Recommended model for beginners: <b>EfficientNetB0</b> (computationally efficient while maintaining strong performance)

Alternatives:
- ResNet50: Great for deep learning with residual connections (preventing gradient issues), but can be heavy on computation for larger datasets
- InceptionV3: Very powerful for complex image features, but slightly more computationally expensive than EfficientNet

In [ ]:
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
df_train_cl = pd.read_pickle("intermediate/train_df_clean.pkl") # replace by cleaned df
display(df_train_cl)

In [ ]:
def get_paths_of_square_images(path):
    new_paths = []
    views = ['front', 'right', 'back', 'left']
    parts = path.replace(".jpg", "").split("/")
    for view in views:
        path = 'train_images_square/' + f"{parts[1]}/{parts[2]}_{view}.jpg"
        new_paths.append(path)
    return new_paths

In [ ]:
get_paths_of_square_images("images/italy/1741683573_45.0502728_7.0621625.jpg")

In [ ]:
square_image_paths = df_train_cl['path'].apply(get_paths_of_square_images).sum()
square_image_paths[0:10]

In [ ]:
# Apply the function to 'path' column and create a new column 'new_paths'
df_train_cl['square_image_paths'] = df_train_cl['path'].apply(get_paths_of_square_images)

# Explode the new column into multiple rows
df_train_cl = df_train_cl.explode('square_image_paths', ignore_index=True)

In [ ]:
display(df_train_cl)

In [ ]:
# Load EfficientNetB0 without the top layers (classification layers)

model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [ ]:
# Create the Sequential model

model_finetune = Sequential()

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),   # Flip images horizontally
    layers.RandomBrightness(0.2),       # Randomly adjust the brightness by up to 20%
    layers.RandomContrast(0.1),
    layers.RandomZoom(0.2)
])

In [ ]:
model_finetune.add(data_augmentation)

In [ ]:
# Add the base EfficientNetB0 model

model_finetune.add(model)

In [ ]:
# Add a global average pooling layer to reduce the dimensions

model_finetune.add(GlobalAveragePooling2D())

In [ ]:
# Add a fully connected dense layer

# model_finetune.add(Dense(1024, activation='relu'))

In [ ]:
# Add a dropout layer to reduce overfitting

# model_finetune.add(Dropout(0.4))

In [ ]:
# model_finetune.add(Dense(256, activation='relu'))
# model_finetune.add(Dropout(0.5))
model_finetune.add(Dense(512, activation='relu'))
model_finetune.add(Dropout(0.4))
model_finetune.add(Dense(128, activation='relu'))
model_finetune.add(Dropout(0.3))

In [ ]:
# Add the output layer with the number of classes you have in your dataset (e.g., countries or regions)

model_finetune.add(Dense(df_train_cl['region_cluster'].nunique(), activation='softmax'))  # Replace num_classes with the number of regions or countries you want to classify

In [ ]:
print(df_train_cl['region_cluster'].nunique())

In [ ]:
# Freeze the EfficientNetB0 base model layers

model_finetune.layers[0].trainable = False

In [ ]:
# Compile the model

model_finetune.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy']) # set learning rate less small?

In [ ]:
# Prepare the data

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) 

In [ ]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train_cl,  # Assuming df has columns 'path' and 'majority_country'
    x_col='square_image_paths',
    y_col='region_cluster',
    target_size=(224, 224),  # Resize images to fit the model
    batch_size=32,
    class_mode='categorical',  # Since it's a multi-class classification problem
    subset='training'  # Set as 'training' or 'validation'
)

In [ ]:
# problem: slices from same panoramic picture can be both in training and validation set -> is this data leakage? 

validation_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train_cl,  
    x_col='square_image_paths',
    y_col='region_cluster',
    target_size=(224, 224),  # Resize images to fit the model
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

In [ ]:
df_train_cl.shape

In [ ]:
# Train the model

history = model_finetune.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),  # Stops early if validation performance doesn't improve
        ModelCheckpoint('best_model.keras', save_best_only=True)  # Save the best model during training
    ]
)

In [ ]:
len(model.layers)

In [ ]:
# Fine-tune the model: Unfreeze some layers

model.trainable = True
for layer in model.layers[:150]:
    layer.trainable = False  # Freeze the first 150 layers

In [ ]:
# Re-compile the model after unfreezing

model_finetune.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Fine-tune the model

history_finetune = model_finetune.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[
        EarlyStopping(patience=3, restore_best_weights=True),  # Stops early if validation performance doesn't improve
        ModelCheckpoint('best_model_ft.keras', save_best_only=True)  # Save the best model during training
    ]
)